In [1]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': '../sst5/train_sst5.csv', 'val' : '../sst5/val_sst5.csv', 'test': '../sst5/test_sst5.csv'})

Using custom data configuration default-1ec86d73e728f420
Reusing dataset csv (C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
train_ds, val_ds, test_ds = dataset["train"], dataset["val"], dataset["test"]

In [3]:
train_ds  

Dataset({
    features: ['sentence', 'label'],
    num_rows: 8544
})

In [4]:
val_ds

Dataset({
    features: ['sentence', 'label'],
    num_rows: 1101
})

In [5]:
test_ds

Dataset({
    features: ['sentence', 'label'],
    num_rows: 2210
})

In [6]:
from fastai.text.all import *
set_seed(42,True)

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length = 60)
 
tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
tokenized_val_ds = val_ds.map(tokenize_function, batched=True)
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-11ef6b7551047f26.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-f932d30016af9069.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-4382cbb478b21b96.arrow


In [9]:
import numpy as np
from datasets import load_metric

acc = load_metric("accuracy")
f1 = load_metric('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    res = {"accuracy" : acc.compute(predictions=predictions, references=labels),
           "f1" : f1.compute(predictions=predictions, references=labels, average="weighted")
           }
        
    return res

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
from fastai.text.all import *
set_seed(42,True)

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length = 60)
 
tokenized_train_ds = train_ds.map(tokenize_function, batched=True)
tokenized_val_ds = val_ds.map(tokenize_function, batched=True)
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-11ef6b7551047f26.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-f932d30016af9069.arrow
Loading cached processed dataset at C:\Users\AR88750\.cache\huggingface\datasets\csv\default-1ec86d73e728f420\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-4382cbb478b21b96.arrow


In [13]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
training_args = TrainingArguments("trainer_evaluation_test", evaluation_strategy="epoch", num_train_epochs=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running training *****
  Num examples = 8544
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2136


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.207000,1.130679,{'accuracy': 0.5149863760217984},{'f1': 0.48721945493637164}
2,0.888000,1.201807,{'accuracy': 0.5113533151680291},{'f1': 0.5024222546546259}


Saving model checkpoint to trainer_evaluation_test\checkpoint-500
Configuration saved in trainer_evaluation_test\checkpoint-500\config.json
Model weights saved in trainer_evaluation_test\checkpoint-500\pytorch_model.bin
Saving model checkpoint to trainer_evaluation_test\checkpoint-1000
Configuration saved in trainer_evaluation_test\checkpoint-1000\config.json
Model weights saved in trainer_evaluation_test\checkpoint-1000\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 1101
  Batch size = 8
Saving model checkpoint to trainer_evaluation_test\checkpoint-1500
Configuration saved in trainer_evaluation_test\checkpoint-1500\config.json
Model weights saved in trainer_evaluation_test\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to trainer_evaluation_test\checkpoint-2000
Configuration saved in trainer_evaluatio

TrainOutput(global_step=2136, training_loss=1.0748390847824039, metrics={'train_runtime': 234.2242, 'train_samples_per_second': 72.956, 'train_steps_per_second': 9.119, 'total_flos': 526894080284160.0, 'train_loss': 1.0748390847824039, 'epoch': 2.0})

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 2210
  Batch size = 8


{'eval_loss': 1.1431002616882324,
 'eval_accuracy': {'accuracy': 0.5271493212669683},
 'eval_f1': {'f1': 0.522969080700315},
 'eval_runtime': 5.8732,
 'eval_samples_per_second': 376.285,
 'eval_steps_per_second': 47.163}

In [18]:
model.save_pretrained('bert_model_finetuned_sst')

Configuration saved in bert_model_finetuned_sst\config.json
Model weights saved in bert_model_finetuned_sst\pytorch_model.bin


In [28]:
sentence_a = test_ds['sentence'][1]
sentence_a

'If you sometimes like to go to the movies to have fun , Wasabi is a good place to start .'

In [25]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert_model_finetuned_sst", num_labels=5)
model_attention = AutoModelForSequenceClassification.from_pretrained("bert_model_finetuned_sst", num_labels=5, output_attentions=True)

In [27]:
def show_head_view(model, tokenizer, sentence_a, sentence_b=None, layer=None, heads=None):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    if sentence_b:
        token_type_ids = inputs['token_type_ids']
        attention = model(input_ids, token_type_ids=token_type_ids)[-1]
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        attention = model(input_ids)[-1]
        sentence_b_start = None
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start, layer=layer, heads=heads)

In [29]:
show_head_view(model_attention, tokenizer, sentence_a)

<IPython.core.display.Javascript object>

In [13]:
'get output'

'get output'

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert_model_finetuned_sst", num_labels=5)

In [47]:
res = {'sentence': [], 'prediction': [],  'label': []}

In [48]:
ind = 100
text = tokenized_test_ds['sentence'][ind]
print(text)
inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
input_ids =inputs['input_ids']
token_type_ids = inputs['token_type_ids']
attention_mask = inputs['attention_mask']

outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask)
pd.DataFrame({'sentence': [text], 'prediction': [torch.argmax(outputs['logits']).item()], 'label' : tokenized_test_ds['label'][ind]})

On the surface , it 's a lovers-on-the-run crime flick , but it has a lot in common with Piesiewicz 's and Kieslowski 's earlier work , films like The Double Life of Veronique .


,sentence,prediction,label
0,"On the surface , it 's a lovers-on-the-run crime flick , but it has a lot in common with Piesiewicz 's and Kieslowski 's earlier work , films like The Double Life of Veronique .",2,3


In [49]:
res['sentence'].append(text)
res['prediction'].append(torch.argmax(outputs['logits']).item())
res['label'].append(tokenized_test_ds['label'][ind])

In [50]:
ind = 0
text = tokenized_test_ds['sentence'][ind]
print(text)
inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
input_ids =inputs['input_ids']
token_type_ids = inputs['token_type_ids']
attention_mask = inputs['attention_mask']

outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask)
pd.DataFrame({'sentence': [text], 'prediction': [torch.argmax(outputs['logits']).item()], 'label' : tokenized_test_ds['label'][ind]})

Effective but too-tepid biopic


,sentence,prediction,label
0,Effective but too-tepid biopic,2,2


In [51]:
res['sentence'].append(text)
res['prediction'].append(torch.argmax(outputs['logits']).item())
res['label'].append(tokenized_test_ds['label'][ind])

In [52]:
ind = 120
text = tokenized_test_ds['sentence'][ind]
print(text)
inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
input_ids =inputs['input_ids']
token_type_ids = inputs['token_type_ids']
attention_mask = inputs['attention_mask']

outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask)
pd.DataFrame({'sentence': [text], 'prediction': [torch.argmax(outputs['logits']).item()], 'label' : tokenized_test_ds['label'][ind]})

An enthralling , playful film that constantly frustrates our desire to know the ` truth ' about this man , while deconstructing the very format of the biography in a manner that Derrida would doubtless give his blessing to .


,sentence,prediction,label
0,"An enthralling , playful film that constantly frustrates our desire to know the ` truth ' about this man , while deconstructing the very format of the biography in a manner that Derrida would doubtless give his blessing to .",4,3


In [53]:
res['sentence'].append(text)
res['prediction'].append(torch.argmax(outputs['logits']).item())
res['label'].append(tokenized_test_ds['label'][ind])

In [55]:
pd.DataFrame(res)

,sentence,prediction,label
0,"On the surface , it 's a lovers-on-the-run crime flick , but it has a lot in common with Piesiewicz 's and Kieslowski 's earlier work , films like The Double Life of Veronique .",2,3
1,Effective but too-tepid biopic,2,2
2,"An enthralling , playful film that constantly frustrates our desire to know the ` truth ' about this man , while deconstructing the very format of the biography in a manner that Derrida would doubtless give his blessing to .",4,3
